# ***QTM 350 Final Project***
### Olivia Song, Elizabeth Bryant, Betty Li, and Aaron Leonard
![Climate](https://www.noaa.gov/sites/default/files/styles/scale_crop_1120x534/public/thumbnails/image/Photo%20collage-climate%20scenarios-global%20climate%20reports-NOAA%20image-landscapes.jpg?itok=rcowMNf5)

For our Data Science Capstone, our group analyzed NOAA Global Historical Climatology Network Daily data from the Registry of Open Data on AWS (https://registry.opendata.aws/noaa-ghcn/). We found this dataset interesting because climate change has been a major topic in the news for the past few years. Also, weather data is faily easy to work with since all the variables are familiar. This dataset contains daily observations for precipitation, temperature, snow depth, and time of observation from multiple resources since 1763. We decided to focus on data from 2019 and 1919, since 2019 is the most current and complete and 1919 is exactly 100 years before. We can compare the data to see how climate has changed over the years. Our work can be found at https://github.com/miaowu128/QTM350FinalProject.git.

We began by looking at the documentation included on the Open Data Registry (https://docs.opendata.aws/noaa-ghcn-pds/readme.html). The readme file gave us insight on the variables we would be looking at. Dr. Jacobson also helped us research some other published notebooks that used the data to see if we could build off of previous work. These were great examples;however, the first notebook used a different file than the one we chose and the second was used to demonstrate how to use various python packages. 

### *Notebook Examples*
- https://livebook.manning.com/book/the-quick-python-book-third-edition/case-study/1
- https://www.kaggle.com/kerneler/starter-aws-open-source-weather-4e419ab3-9

# ***Data Architecture***
Throughout this semester we have learned about many AWS services that can be useful for running big data. Some of the services we used for our project are: Registry of Open Data, s3, EC2 instance, and SageMaker. Using draw.io, we created a diagram to show our workflow.
![Data Architecture](QTM350_FinalProject.png)

# ***Cleaning & Analysis***

## **Elizabeth**

First, we started by accessing the data from the Open Data Registry using the Amazon Resource Name and creating our own s3 bucket. We used the exploring-data notebook from the aws-open-data-analytics-notebooks repository to set up our project (https://github.com/aws-samples/aws-open-data-analytics-notebooks).

In [ ]:
# Setting up environment
import boto3
import botocore
import pandas as pd
from IPython.display import display, Markdown

In [ ]:
# Using boto3 to access s3 API
s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')

In [ ]:
# Creating a unique s3 bucket
def create_bucket(bucket):
    import logging

    try:
        s3.create_bucket(Bucket=bucket)
    except botocore.exceptions.ClientError as e:
        logging.error(e)
        return 'Bucket ' + bucket + ' could not be created.'
    return 'Created or already exists ' + bucket + ' bucket.'

#### Note
Each group member made their own s3 bucket to work with since s3 bucket names have to be unique universally. This unique bucket is useful for the user to save the data they are working with and access it later.

In [ ]:
create_bucket('open-data-analytics-noaa')

In [ ]:
# List bucket to confirm that it is created
def list_buckets(match=''):
    response = s3.list_buckets()
    if match:
        print(f'Existing buckets containing "{match}" string:')
    else:
        print('All existing buckets:')
    for bucket in response['Buckets']:
        if match:
            if match in bucket["Name"]:
                print(f'  {bucket["Name"]}')

In [ ]:
list_buckets(match='noaa')

In [ ]:
# List bucket contents: Allows us to access and see the contents of the bucket that contains the data we want
def list_bucket_contents(bucket, match='', size_mb=0):
    bucket_resource = s3_resource.Bucket(bucket)
    total_size_gb = 0
    total_files = 0
    match_size_gb = 0
    match_files = 0
    for key in bucket_resource.objects.all():
        key_size_mb = key.size/1024/1024
        total_size_gb += key_size_mb
        total_files += 1
        list_check = False
        if not match:
            list_check = True
        elif match in key.key:
            list_check = True
        if list_check and not size_mb:
            match_files += 1
            match_size_gb += key_size_mb
            print(f'{key.key} ({key_size_mb:3.0f}MB)')
        elif list_check and key_size_mb <= size_mb:
            match_files += 1
            match_size_gb += key_size_mb
            print(f'{key.key} ({key_size_mb:3.0f}MB)')

    if match:
        print(f'Matched file size is {match_size_gb/1024:3.1f}GB with {match_files} files')            
    
    print(f'Bucket {bucket} total size is {total_size_gb/1024:3.1f}GB with {total_files} files')

In [ ]:
# Lists the files in the NOAA Global Historical Climatology Network Daily data
list_bucket_contents(bucket='noaa-ghcn-pds', match='.csv', size_mb= 1000)

In [ ]:
# Previewing the csv files that contain the data
def preview_csv_dataset(bucket, key, rows=10):
    data_source = {
            'Bucket': bucket,
            'Key': key
        }
    # Generate the URL to get Key from Bucket
    url = s3.generate_presigned_url(
        ClientMethod = 'get_object',
        Params = data_source
    )

    data = pd.read_csv(url, nrows=rows, header = None)
    return data

The data we chose was from 2019 and 1919, so we can compare the climate 100 years apart. The `rows` argument allows us to set the number of rows we would like to see in the preview. These Open Data Registry datasets can contain millions of rows, so this argument is very useful to see a preview of what we're analyzing.

In [ ]:
df_2019 = preview_csv_dataset(bucket='noaa-ghcn-pds', key='csv/2019.csv', rows = 1000)

In [ ]:
df_1919 = preview_csv_dataset(bucket='noaa-ghcn-pds', key='csv/1919.csv', rows = 1000)

Now that we have chosen our datasets, we can copy our file into the bucket we created earlier.

In [ ]:
# Check and see if file exists in the bucket we created earlier. Then copy the data from NOAA into our own bucket
def key_exists(bucket, key):
    try:
        s3_resource.Object(bucket, key).load()
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            # The key does not exist.
            return(False)
        else:
            # Something else has gone wrong.
            raise
    else:
        # The key does exist.
        return(True)

def copy_among_buckets(from_bucket, from_key, to_bucket, to_key):
    if not key_exists(to_bucket, to_key):
        s3_resource.meta.client.copy({'Bucket': from_bucket, 'Key': from_key}, 
                                        to_bucket, to_key)        
        print(f'File {to_key} saved to S3 bucket {to_bucket}')
    else:
        print(f'File {to_key} already exists in S3 bucket {to_bucket}') 

In [ ]:
copy_among_buckets(from_bucket='noaa-ghcn-pds', from_key='csv/2019.csv',to_bucket='open-data-analytics-noaa', to_key='NOAA_2019.csv')

In [ ]:
copy_among_buckets(from_bucket='noaa-ghcn-pds', from_key='csv/1919.csv',to_bucket='open-data-analytics-noaa', to_key='NOAA_1919.csv')

To confirm that the files were copied over, I went back to my s3 to make sure the files were in the bucket. As you can see the 2019 file, is 1.1 GB which is HUGE. This is why we access s3 buckets for data instead of saving the files onto our device. Downloading files that size are sometimes too big to work with on our own machines. 
![s3](NOAA_s3.png)

To begin cleaning our data, you can see that our data does not have column names. The documentation lists what each column means (https://docs.opendata.aws/noaa-ghcn-pds/readme.html). We can begin by assigning column names to make it easier to read.

In [ ]:
#Checking to see what columns look like
df_2019.head()

#Renaming Columns
df_2019.columns = ["ID","Date","Element","Element_Value","MFlag","QFlag","SFlag","Obs_Time"]

#Checking to see new columns
df_2019

In [ ]:
#Checking to see what columns look like
df_1919.head()

#Renaming Columns
df_1919.columns = ["ID","Date","Element","Element_Value","MFlag","QFlag","SFlag","Obs_Time"]

#Checking to see new columns
df_1919

Great! We have real columns now. The 'Element' column is all mixed up with different weather descriptions. With each element input, there is a value or measurement for that element in the Element_Value column. I think it would be interesting to see the averages for each element for the year of 2019, by using the `groupby` and `.mean()` argument.

In [ ]:
df_2019_Element = df_2019.groupby(by = ['Element'])['Element_Value'].mean()
df_2019_Element

In [ ]:
df_1919_Element = df_1919.groupby(by = ['Element'])['Element_Value'].mean()
df_1919_Element

The five main elements are precipitation(PRCP), snowfall(SNOW), snow depth(SNWD), maximum temperature(TMAX), and minimum temperature(TMIN). However, there are different subcategories of each, and these descriptions can be found in the documention. 

The list of outputs is kind of difficult to look at, so it would be better to read in a data visualization.

In [ ]:
plot = df_2019_Element.plot.bar(x='Element', y = 'Element_Value',bottom = 0.5)
plot.set_ylabel('Element Value')
plot.set_title('Average Measure of Elements - 2019')

Peak Gust Time(PGTM) has the highest average measurement of 1505.250000, which is measured as time in hours and minutes.The lowest average measurement is Temperature at the Time of Observation(TOBS) at -41.043478, which is measured in celsius.

In [ ]:
plot = df_1919_Element.plot.bar(x='Element', y = 'Element_Value',bottom = 0.5)
plot.set_ylabel('Element Value')
plot.set_title('Average Measure of Elements - 1919')

The Peak Gust Time(PGTM) was not recorded in 1919. This could be because over the years data collection has become more precise and has extended to greater categories. However we can compare the Temperature at the Time of Observation(TOBS). In 1919, the average for TOBS was -6.304878 celsius. This a 34.7386 degree difference between 2019 and 1919, which shows that weather has greatly changed over the years. 

## **Aaron Leonard**

If you don't want a graph, you can simply run an argument, that takes the csv and aggregates it down to a single number.

In [ ]:
#We want to find a summary of all the maximum temperatures in the dataset for 2019, but I do not want a graph.
#I want just a simple numeric answer. 

df_2019Hot= df_2019[df_2019['Element']=='TMAX']
#Now for the mean, meadian, and mode. 
print("The median Max temperature is:", df_2019Hot["Element_Value"].median())
print("The average Max temperature is:", df_2019Hot["Element_Value"].mean())
print("The mode Max temperature is:", df_2019Hot["Element_Value"].mode())

print("------")

#Now for minimum temperature:
df_2019Cold= df_2019[df_2019['Element']=='TMIN']

print("The median Min temperature is:", df_2019Cold["Element_Value"].median())
print("The average Min temperature is:", df_2019Cold["Element_Value"].mean())
print("The mode Min temperature is:", df_2019Cold["Element_Value"].mode())

In [ ]:
#Now for 1919:
#First max temperature:
df_1919Hot= df_1919[df_1919['Element']=='TMAX']

print("The median Max temperature is:", df_1919Hot["Element_Value"].median())
print("The average Max temperature is:", df_1919Hot["Element_Value"].mean())
print("The mode Max temperature is:", df_1919Hot["Element_Value"].mode())

print("------")
#Now for minimum temperature:
df_1919Cold= df_1919[df_2019['Element']=='TMIN']

print("The median Min temperature is:", df_1919Cold["Element_Value"].median())
print("The average Min temperature is:", df_1919Cold["Element_Value"].mean())
print("The mode Min temperature is:", df_1919Cold["Element_Value"].mode())

The average is basically at every country in the dataset, and each region.
As you can see, it appears that the maximum average world temperature has gone up since 1919.
It has gone up from 23.31 to 67.8 Celcius. 
Average minimum temperature has also gone down from -40 to 4.07 degrees Celcius. 
Why this is is entirely speculative, since this is only looking at two years, we don't know when the turning point for temperature change was.
It could have been during World War II when carbon emissions increased globally or it could have been during the sixties and seventies.
Its unknown when the turning point was. 
What is clear is that world temperatures are reaching higher extremes.

## **Olivia Song**

The dataset contains an observation time variable that is set up as yyyy/mm/dd. It would be easier to read if it could be split up. 

In [ ]:
#Create a new function that reads the whole file
def preview_csv_dataset_all(bucket, key):

    data_source = {
            'Bucket': bucket,
            'Key': key
        }
    # Generate the URL to get Key from Bucket
    url = s3.generate_presigned_url(
        ClientMethod = 'get_object',
        Params = data_source
    )
    data = pd.read_csv(url) # read all rows
    return data

In [ ]:
# read 1919 dataset
df_1919_all = preview_csv_dataset_all(bucket='noaa-ghcn-pds', key='csv/1919.csv')

In [ ]:
#Renaming Columns
df_1919_all.columns = ["ID","Date","Element","Element_Value","MFlag","QFlag","SFlag","Obs_Time"]

#Checking to see new columns
df_1919_all.head()

In [ ]:
df_1919_all.describe()

## **Datetime Package**
You might have noticed that the "Date" column is in integer type, which will cause some trouble if we want to plot a time series graph. Analyzing datasets with dates and times is often very cumbersome. Months of different lengths, different distributions of weekdays and weekends, leap years, and the dreaded timezones are just a few things you may have to consider depending on your context. For this reason, Python has a data type specifically designed for dates and times called datetime. In this specific case, you may want to convert "Date" colulmn from integer type to datetime type.
We first import the package.


In [ ]:
from datetime import datetime, timedelta

Then we write a function that can help us to convert the datetime.

In [ ]:
def toDatetime(x):
    x = str(x) # convert the integer type to string.
    date = datetime(year=int(x[0:4]), month=int(x[4:6]), day=int(x[6:8])) # cut the string into pieces and convert them to datetime type.
    return date

Since we want to apply the function to all rows of "Date" column, we have to convert the dateset to padas datafreme and then use ".apply()" to apply the function.

In [ ]:
df_1919_pd = pd.DataFrame(df_1919_all) 
df_1919_pd['Date'] = df_1919_pd['Date'].apply(toDatetime)

In [ ]:
df_1919_date = df_1919_pd

Let's look at the new "Date" column.

In [ ]:
df_1919_date.head(20)

Now it's in datetime type!

## **Indexing pandas DataFrames with DatetimeIndex**
After you've converted the date column to the datetime format, it is usually a good idea to index your DataFrame by the date, creating the DatetimeIndex. Like the datetime type, the DatetimeIndex is a special index type designed to work with dates and times. By using the .set_index() method with the inplace argument set equal to True, you can remove the date column from your dataset and append it as the index of your DataFrame:

In [ ]:
# Set index
df_1919_date.set_index('Date', inplace=True)

In [ ]:
df_1919_date[df_1919_date['Element'] == "TMAX", ].Element_Value.plot(title = 'test')

In [ ]:
plot = df_2019_pd.plot.bar(x='Element', y = 'Element_Value',bottom = 0.5)
plot.set_ylabel('Element Value')
plot.set_title('Average Measure of Elements - 2019')

# ***Themes and Extensions***
## **Alexa**
Themes can be a very good way to engage the reader further, by providing different colors and fonts to break up the seas of white in the default Jupyter Notebook theme. We'll be doing so by using Jupyter Themes, a program created by Kyle Donovan, whose Github can be found here: https://github.com/dunovank/jupyter-themes. This Github helps provide basic information about setting up Jupyter Themes, as well as trouble-shooting. To get started, we need to install jupyterthemes, and install and update packages that we need up to date.

In [ ]:
!pip install -U jupyter

In [ ]:
!pip install sparkmagic

In [ ]:
!pip install jupyterthemes

## **Theme**
Now let's pick a theme. To see the full list, type jt -l in the command line. You'll see plenty of themes, and tofind out which one works best for you, take a look and mess around! We decided on solarizedl. It has a creme background with red lettering.

In [ ]:
!jt -l

## **Font and Size**
Jupyterthemes allow for so much more than just changing the themes. It can also extend to fonts, font size, text, and more. Below you can see their whole command line options. For this notebook, we'll be experimenting with fonts for the different cells and their sizes. In this notebook, we'll be using the solarizedl theme, Source Code Pro for the code cell font, Source Sans for the notebook font, a notebook font of size 11, a text cell font of Crimson Serif, the text cell font size to 11, and the Output Area fontsize to 8.

In [ ]:
!jt -t solarizedl -f sourcemed -nf sourcesans -nfs 11 -tf crimsonserif -tfs 11 -ofs 8

After inputting your preferences, you've made the biggest changes to the aesthetics! The reader will have plenty of visual interest, but what about readibility? Functionality? The last step we'll do is add notebook extensions to increase functionality on both ends--for the reader and creators.

## **Extensions**
Jupyter notebooks are great a great tool to use for a variety of reasons, especially considering it was made to have be an extendable environment. But what are extensions? They're simple tools usually coded in Javascript which can be added on to notebooks to increase their functionality. Some of the extensions that we'll be using include spellchecking and table of contents. First, we need to install and enable the extension manager, since its default is turned off.

In [ ]:
!pip install https://github.com/ipython-contrib/jupyter_contrib_nbextensions/tarball/master

In [ ]:
!jupyter contrib nbextension install --user

In [ ]:
!jupyter nbextension enable codefolding/main

In [ ]:
!jupyter nbextension enable spellchecker/main

In [ ]:
!jupyter nbextension enable table_beautifier/main